In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
from talus_data_analysis.plot import histogram
from talus_data_analysis.elib import Elib
from talus_data_analysis.load import read_df_from_s3
from talus_data_analysis.save import write_df_to_s3
from dotenv import load_dotenv
import tempfile
import sqlite3
import math
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [4]:
load_dotenv()

True

In [5]:
gauth = GoogleAuth(settings_file="../settings.yaml")

In [14]:
ENCYCLOPEDIA_BUCKET = "talus-data-pipeline-encyclopedia-bucket"
DATA_FOLDER = "../data/210521_THP1"
S3_FOLDER = "wide/210521_THP1"
PROJECT_NAME = "THP-1 Screening"
QUANT_ELIB_NAME = "RESULTS"

ELIB_FILE = "RESULTS-quant.elib"
peptide_protein_file = "peptide_proteins_results.csv"
peptide_protein_norm_output = "peptide_proteins_normalized.csv"
msstats_groupcompare_output = "msstats_groupcompare.csv"
comparison_matrix_file = "comparison_matrix.csv"

In [15]:
peptide_df = read_df_from_s3(bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{QUANT_ELIB_NAME}-quant.elib.peptides.txt", inputformat="txt")
peptide_df = peptide_df.drop("numFragments", axis=1)
# make sure there is one protein per column
peptide_df = peptide_df.drop("Protein", axis=1).join(peptide_df["Protein"].str.split(";", expand=True).stack().reset_index(drop=True, level=1).rename("Protein"))
# melt the dataframe so that each source file is in a seperate row instead of column
peptide_df = peptide_df.melt(id_vars=["Peptide", "Protein"], var_name="SourceFile", value_name="TotalIntensity")
peptide_df["Run"] = peptide_df["SourceFile"].apply(lambda x: x.split(".")[0].split("_")[-1])

In [16]:
peptide_df.tail()

,Peptide,Protein,SourceFile,TotalIntensity,Run
2812295,YYVTIIDAPGHR,sp|Q5VTE0|EF1A3_HUMAN,210525_talus_99.mzML,0.0,99
2812296,YYVTIIDAPGHR,sp|P68104|EF1A1_HUMAN,210525_talus_99.mzML,0.0,99
2812297,YYVTIIDAPGHRDFIK,sp|Q5VTE0|EF1A3_HUMAN,210525_talus_99.mzML,0.0,99
2812298,YYVTIIDAPGHRDFIK,sp|P68104|EF1A1_HUMAN,210525_talus_99.mzML,0.0,99
2812299,YYYAVVDC[+57.021464]DSPETASK,sp|Q9H501|ESF1_HUMAN,210525_talus_99.mzML,0.0,99


## Template DF

In [19]:
sample_df = pd.read_excel("~/Downloads/MS_samples_May2021.xlsx")
sample_df = sample_df[(sample_df["Project"] == PROJECT_NAME) & (sample_df["Comparison"].notna())]
sample_df["Sample Description"] = sample_df["Sample Name"]
sample_df["Run"] = sample_df["Run ID"].apply(lambda x: x.split("_")[-1])
sample_df = sample_df.rename(columns={"Sample Name": "BioReplicate", "Sample Description": "Condition"})
sample_df = sample_df[["Run", "BioReplicate", "Condition", "Comparison"]]

In [20]:
sample_df

,Run,BioReplicate,Condition,Comparison
41,33,THP1_EP1_S1,THP1_EP1_S1,Control
42,34,THP1_EP1_S2,THP1_EP1_S2,THP1_EP1_S1
43,35,THP1_EP1_S3,THP1_EP1_S3,THP1_EP1_S1
44,36,THP1_EP1_S4,THP1_EP1_S4,THP1_EP1_S1
45,37,THP1_EP1_S5,THP1_EP1_S5,THP1_EP1_S1
...,...,...,...,...
126,116,THP1_KP3_S8,THP1_KP3_S8,THP1_KP3_S1
127,117,THP1_KP3_S9,THP1_KP3_S9,THP1_KP3_S1
128,118,THP1_KP3_S10,THP1_KP3_S10,THP1_KP3_S1
129,119,THP1_KP3_S11,THP1_KP3_S11,THP1_KP3_S1


In [21]:
msstats_df = pd.merge(peptide_df, sample_df, how="right", on="Run")

In [22]:
## Add a few required columns and rename header to match MSstats convention
msstats_df = msstats_df.drop(["Run", "Comparison"], axis=1)
msstats_df["PrecursorCharge"] = 2
msstats_df["IsotopeLabelType"] = "L"
msstats_df["FragmentIon"] = "y0"
msstats_df["ProductCharge"] = "1"
msstats_df = msstats_df.rename(columns={"Peptide": "PeptideSequence",
                                        "Protein": "ProteinName",
                                        "SourceFile": "Run",
                                        "TotalIntensity": "Intensity"})
msstats_df = msstats_df.drop_duplicates()
msstats_df = msstats_df.dropna(subset=["PeptideSequence", "ProteinName"])

In [23]:
msstats_df = msstats_df.sort_values(by=["PeptideSequence", "Intensity"]).reset_index(drop=True)

In [24]:
msstats_df.tail()

,PeptideSequence,ProteinName,Run,Intensity,BioReplicate,Condition,PrecursorCharge,IsotopeLabelType,FragmentIon,ProductCharge
2305261,YYYAVVDC[+57.021464]DSPETASK,sp|Q9H501|ESF1_HUMAN,210525_talus_39.mzML,151961.42,THP1_IAA,THP1_IAA,2,L,y0,1
2305262,YYYAVVDC[+57.021464]DSPETASK,sp|Q9H501|ESF1_HUMAN,210525_talus_52.mzML,401982.12,THP1_EP2_S12,THP1_EP2_S12,2,L,y0,1
2305263,YYYAVVDC[+57.021464]DSPETASK,sp|Q9H501|ESF1_HUMAN,210525_talus_45.mzML,462787.20,THP1_EP2_S5,THP1_EP2_S5,2,L,y0,1
2305264,YYYAVVDC[+57.021464]DSPETASK,sp|Q9H501|ESF1_HUMAN,210525_talus_44.mzML,559919.20,THP1_EP2_S4,THP1_EP2_S4,2,L,y0,1
2305265,YYYAVVDC[+57.021464]DSPETASK,sp|Q9H501|ESF1_HUMAN,210525_talus_49.mzML,693205.00,THP1_EP2_S9,THP1_EP2_S9,2,L,y0,1


In [25]:
msstats_df.to_csv(f"{DATA_FOLDER}/{peptide_protein_file}")

KeyboardInterrupt: 

In [ ]:
write_df_to_s3(dataframe=msstats_df, bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{peptide_protein_file.replace('.csv', '.parquet')}", outputformat="parquet")

In [ ]:
def get_comparison_matrix(df, filter_target_func=lambda x:x, leave_out=set()):
    df = df.sort_values(by="Condition")
    dmso_map = {condition: dmso for (condition, dmso) in zip(df["Condition"], df["Comparison"])}
    
    comp_lol = []
    targets = sorted(dmso_map.keys())
    targets = [t for t in targets if t not in leave_out]
    comp_df_index = []
    for i, s in enumerate(targets):
        comp_list = [0 for t in targets]
        if dmso_map[s] in dmso_map:
            # make dmso 0
            comp_list[targets.index(dmso_map[s])] = -1
            # make target itself 1
            comp_list[i] = 1

            comp_lol.append(comp_list)
            comp_df_index.append(f"{filter_target_func(s)}/{dmso_map[s]}")

    comp_df = pd.DataFrame(comp_lol)
    comp_df.index = comp_df_index
    
    return comp_df

In [ ]:
comp_matrix = get_comparison_matrix(df=sample_df)

In [ ]:
comp_matrix

In [ ]:
comp_matrix.to_csv(f"{DATA_FOLDER}/{comparison_matrix_file}")

# Run R Script (MSStats) ...

## Write msstats normalized peptide protein df to s3

In [7]:
msstats_df_norm = pd.read_csv(f"{DATA_FOLDER}/{peptide_protein_norm_output}")

In [8]:
msstats_df_norm

,PROTEIN,PEPTIDE,TRANSITION,FEATURE,LABEL,GROUP_ORIGINAL,SUBJECT_ORIGINAL,RUN,GROUP,SUBJECT,INTENSITY,SUBJECT_NESTED,ABUNDANCE,FRACTION,originalRUN,censored
0,sp|A0A0B4J271|TVAL3_HUMAN,YISLFIR_2,y0_1,YISLFIR_2_y0_1,L,THP1_EP1_S1,THP1_EP1_S1,1,1,1,1.00,1.10,0.000000,1,210525_talus_33.mzML,True
1,sp|A0AV96|RBM47_HUMAN,LLGVC[+57.021464]C[+57.021464]SVDNC[+57.021464...,y0_1,LLGVC[+57.021464]C[+57.021464]SVDNC[+57.021464...,L,THP1_EP1_S1,THP1_EP1_S1,1,1,1,1.00,1.10,0.000000,1,210525_talus_33.mzML,True
2,sp|A0AV96|RBM47_HUMAN,SFGQFNPGC[+57.021464]VER_2,y0_1,SFGQFNPGC[+57.021464]VER_2_y0_1,L,THP1_EP1_S1,THP1_EP1_S1,1,1,1,242559.73,1.10,17.887981,1,210525_talus_33.mzML,False
3,sp|A0AVK6|E2F8_HUMAN,IESVNVAPENAGTQQGR_2,y0_1,IESVNVAPENAGTQQGR_2_y0_1,L,THP1_EP1_S1,THP1_EP1_S1,1,1,1,1.00,1.10,0.000000,1,210525_talus_33.mzML,True
4,sp|A0AVT1|UBA6_HUMAN,TVFFESLER_2,y0_1,TVFFESLER_2_y0_1,L,THP1_EP1_S1,THP1_EP1_S1,1,1,1,1.00,1.10,0.000000,1,210525_talus_33.mzML,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1842945,sp|Q9Y6X9|MORC2_HUMAN,TASRPAPLVQQLSPSLLPNSK_2,y0_1,TASRPAPLVQQLSPSLLPNSK_2_y0_1,L,THP1_MP1_S9,THP1_MP1_S9,82,82,68,1.00,82.68,0.000000,1,210525_talus_89.mzML,True
1842946,sp|Q9Y6X9|MORC2_HUMAN,TFHEEEGIDEVIVPLPTWNAR_2,y0_1,TFHEEEGIDEVIVPLPTWNAR_2_y0_1,L,THP1_MP1_S9,THP1_MP1_S9,82,82,68,1.00,82.68,0.000000,1,210525_talus_89.mzML,True
1842947,sp|Q9Y6X9|MORC2_HUMAN,TNIVALLQK_2,y0_1,TNIVALLQK_2_y0_1,L,THP1_MP1_S9,THP1_MP1_S9,82,82,68,1.00,82.68,0.000000,1,210525_talus_89.mzML,True
1842948,sp|Q9Y6X9|MORC2_HUMAN,TREPVTDNVEK_2,y0_1,TREPVTDNVEK_2_y0_1,L,THP1_MP1_S9,THP1_MP1_S9,82,82,68,1.00,82.68,0.000000,1,210525_talus_89.mzML,True


In [9]:
write_df_to_s3(dataframe=msstats_df_norm, bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{peptide_protein_norm_output.replace('.csv', '.parquet')}", outputformat="parquet")

## Write msstats groupcompare df to s3

In [10]:
msstats_groupcompare = pd.read_csv(f"{DATA_FOLDER}/{msstats_groupcompare_output}")

In [11]:
msstats_groupcompare

,Protein,Label,log2FC,SE,Tvalue,DF,pvalue,adj.pvalue,issue,MissingPercentage,ImputationPercentage
0,sp|A0A0B4J271|TVAL3_HUMAN,THP1_EP1_S2/THP1_EP1_S1,NaN,NaN,NaN,NaN,NaN,NaN,completeMissing,1.000000,0.000000
1,sp|A0AV96|RBM47_HUMAN,THP1_EP1_S2/THP1_EP1_S1,-inf,NaN,NaN,NaN,NaN,0.0,oneConditionMissing,0.750000,0.250000
2,sp|A0AVK6|E2F8_HUMAN,THP1_EP1_S2/THP1_EP1_S1,NaN,NaN,NaN,NaN,NaN,NaN,completeMissing,1.000000,0.000000
3,sp|A0AVT1|UBA6_HUMAN,THP1_EP1_S2/THP1_EP1_S1,NaN,NaN,NaN,NaN,NaN,NaN,completeMissing,1.000000,0.000000
4,sp|A0FGR8|ESYT2_HUMAN,THP1_EP1_S2/THP1_EP1_S1,-inf,NaN,NaN,NaN,NaN,0.0,oneConditionMissing,0.923077,0.423077
...,...,...,...,...,...,...,...,...,...,...,...
300361,sp|Q9Y6V7|DDX49_HUMAN,THP1_MP1_S9/THP1_KP5_S1,-inf,NaN,NaN,NaN,NaN,0.0,oneConditionMissing,0.833333,0.333333
300362,sp|Q9Y6W5|WASF2_HUMAN,THP1_MP1_S9/THP1_KP5_S1,-inf,NaN,NaN,NaN,NaN,0.0,oneConditionMissing,0.500000,0.000000
300363,sp|Q9Y6X3|SCC4_HUMAN,THP1_MP1_S9/THP1_KP5_S1,inf,NaN,NaN,NaN,NaN,0.0,oneConditionMissing,0.500000,0.000000
300364,sp|Q9Y6X8|ZHX2_HUMAN,THP1_MP1_S9/THP1_KP5_S1,NaN,NaN,NaN,NaN,NaN,NaN,completeMissing,1.000000,0.000000


In [12]:
write_df_to_s3(dataframe=msstats_groupcompare, bucket=ENCYCLOPEDIA_BUCKET, key=f"{S3_FOLDER}/{msstats_groupcompare_output.replace('.csv', '.parquet')}", outputformat="parquet")